In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Irritable_bowel_syndrome_(IBS)/GSE66824'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genomic and Clinical Effects Associated with a Relaxation Response Mind-Body Intervention in Patients with Irritable Bowel Syndrome and Inflammatory Bowel Disease"
!Series_summary	"Patients with chronic illnesses such as Irritable Bowel Syndrome (IBS) or Inflammatory Bowel Disease (IBD) often have reduced quality of life. IBS is characterized by abdominal pain/discomfort associated with altered bowel function, such as diarrhea or constipation, without gross structural changes or inflammation [1]; IBD is characterized by gross inflammation in the gastrointestinal (GI) tract which can result in symptoms such as abdominal pain, cramping, diarrhea and bloody stools.  IBS and IBD can profoundly affect quality of life and are influenced by stress and resiliency.The impact of mind-body interventions (MBIs) on IBS and IBD patients has not previously been examined. In this study IBS and  IBD patients were enrolled in a 9-week relaxation response based mind

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Since the dataset involves transcriptional expression profiling, set this to True

# Definitions for trait, age, and gender rows
trait_row = 1 if 'disease state: IBS' in sample_characteristics_dict[1] else None
age_row = None  # age information not present in the sample characteristics dictionary
gender_row = None  # gender information also not present in the sample characteristics dictionary

# Conversion functions
def convert_trait(value):
    # e.g., 'disease state: IBS' -> 'IBS'
    mapped_value = value.split(': ')[-1].strip()
    if mapped_value == 'IBS':
        return 1  # trait is binary where IBS is 1
    elif mapped_value == 'IBD':
        return 0  # trait is binary where IBD is 0
    return None

def convert_age(value):
    return None  # No age information to convert

def convert_gender(value):
    return None  # No gender information to convert

# Save cohort information
save_cohort_info('GSE66824', './preprocessed/Irritable_bowel_syndrome_(IBS)/cohort_info.json', is_gene_available, trait_row is not None)

# Extract clinical features
clinical_data_defined = trait_row is not None
if clinical_data_defined:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Irritable_bowel_syndrome_(IBS)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/trait_data/GSE66824.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1632703': [1], 'GSM1632704': [1], 'GSM1632705': [1], 'GSM1632706': [1], 'GSM1632707': [1], 'GSM1632708': [1], 'GSM1632709': [1], 'GSM1632710': [1], 'GSM1632711': [1], 'GSM1632712': [1], 'GSM1632713': [1], 'GSM1632714': [1], 'GSM1632715': [1], 'GSM1632716': [1], 'GSM1632717': [1], 'GSM1632718': [1], 'GSM1632719': [1], 'GSM1632720': [1], 'GSM1632721': [1], 'GSM1632722': [1], 'GSM1632723': [1], 'GSM1632724': [1], 'GSM1632725': [1], 'GSM1632726': [1], 'GSM1632727': [1], 'GSM1632728': [1], 'GSM1632729': [1], 'GSM1632730': [1], 'GSM1632731': [0], 'GSM1632732': [0], 'GSM1632733': [0], 'GSM1632734': [0], 'GSM1632735': [0], 'GSM1632736': [0], 'GSM1632737': [0], 'GSM1632738': [0], 'GSM1632739': [0], 'GSM1632740': [0], 'GSM1632741': [0], 'GSM1632742': [0], 'GSM1632743': [0], 'GSM1632744': [0], 'GSM1632745': [0], 'GSM1632746': [0], 'GSM1632747': [0], 'GSM1632748': [0], 'GSM1632749': [0], 'GSM1632750': [0], 'GSM1632751': [0], 'GSM1632752': [0], 'GSM1632753': [0], 'GSM1632754': [0], 'GSM1632755

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the identifier_key and gene_symbol_key based on gene annotation preview
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/gene_data/GSE66824.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, merged_data = judge_and_remove_biased_features(merged_data, 'Irritable_bowel_syndrome_(IBS)')

# 4. Save the cohort information.
save_cohort_info('GSE66824', './preprocessed/Irritable_bowel_syndrome_(IBS)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Irritable_bowel_syndrome_(IBS)/GSE66824.csv'
    merged_data.to_csv(csv_path)


For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '1.0' with 28 occurrences. This represents 42.42% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.

